In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC


In [7]:
# Set up Selenium WebDriver
driver = webdriver.Chrome()  # Ensure chromedriver is in PATH

# Open the target website
# IMPORTANT: This scraper relies on the current structure (HTML/DOM layout) of the page.
# If the site is updated or built differently, the code may not work without adjustments.
url = 'https://fangstjournalen.dtu.dk/fangst.nsf/iframe.xsp?open&assoc=546F2600F0210439C12585450030B71E&type=1'  # Replace with the URL you wish to scrape
driver.get(url)
time.sleep(5)  # Wait for the page to fully load (adjust if necessary)

In [6]:
# List to accumulate all years' data
all_data = []

# Locate the dropdown menu element and get all options
dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sYear"]')))
select = Select(dropdown)
years = [option.get_attribute("value") for option in select.options if option.get_attribute("value")]

# Loop through each year option in the dropdown menu
for year in years:
    # Skip the "-" option (which might represent "All years" or an empty selection)
    if year == "-":
        continue
    
    # Select the year in the dropdown menu
    select.select_by_value(year)
    print(f"Selected year: {year}")

    # Wait for the table to reload or data to refresh after selecting a year
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="table_div"]/div/div/table/tbody')))

    # Locate the table body
    table_body = driver.find_element(By.XPATH, '//*[@id="table_div"]/div/div/table/tbody')

    # Initialize a list to store each row of data for the current year
    data = []

    # Loop through each row in the table body
    rows = table_body.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        # Get all cells in the row
        cells = row.find_elements(By.TAG_NAME, 'td')
        # Extract text from each cell and handle any missing values with a default value 'N/A'
        row_data = [cell.text.strip() if cell.text.strip() != '' else 'N/A' for cell in cells]
        data.append(row_data)

    # Append the current year's data to all_data
    all_data.extend(data)

# Close the browser
driver.quit()

# Define column names based on table structure
column_names = ['Date', 'Species', 'Water Body', 'Zone', 'Name', 'Length', 'Weight', 
                'Bait', 'Retained', 'Sex', 'Adipose fin clipped', 'Photo']

# Convert all_data to a DataFrame
df = pd.DataFrame(all_data, columns=column_names)

# Save the DataFrame to a CSV file
df.to_csv('holested_sneum-lystfisker.csv', index=False)
print("Data saved to fish_data_all_years.csv")


Selected year: 2025
Selected year: @
Data saved to fish_data_all_years.csv
